In [ ]:

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.svm import SVC


In [ ]:

df = pd.read_csv("train.csv")

id_values = df["ID"].copy()
df = df.drop(["ID"], axis=1)
df = df.fillna("Miss")

df.head()


In [ ]:

def to_onehot(x):
    values = np.unique(x)
    result = np.zeros((len(x), len(values)))
    for i, val in enumerate(values):
        result[:, i] = (x == val).astype(int)
    return result, values

def replace_column_with_onehot(df, col):
    onehot, values = to_onehot(df[col].values)
    df = df.drop([col], axis=1)
    for i, v in enumerate(values):
        df[f"{col}_{v}"] = onehot[:, i]
    return df


In [ ]:

categorical_cols = df.select_dtypes(include=["object"]).columns

df_encoded = df.copy()
for col in categorical_cols:
    df_encoded = replace_column_with_onehot(df_encoded, col)

df_encoded.head()


In [ ]:

target_col = "TARGET"

X = df_encoded.drop([target_col], axis=1)
y = df_encoded[target_col]


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:

model = SVC(kernel="rbf", probability=True, C=1, gamma="scale", random_state=42)

model.fit(X_train_scaled, y_train)


In [ ]:

y_pred = model.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))


In [ ]:

import pickle

with open("modelo_03_SVM.pkl", "wb") as f:
    pickle.dump(model, f)

print("Modelo guardado como modelo_03_SVM.pkl")
